## Problems in practice
You now know most of what you need to tackle data that includes dates and times, but there are a few other problems you might encounter in practice. In this final chapter you'll learn a little more about these problems by returning to some of the earlier data examples and learning how to handle time zones, deal with times when you don't care about dates, parse dates quickly, and output dates and times.

### Setting the timezone
If you import a datetime and it has the wrong timezone, you can set it with force_tz()# . 
Pass in the datetime as the first argument and the appropriate timezone to the tzone argument. # 
Remember the timezone needs to be one from OlsonNames().
I wanted to watch New Zealand in the Women's World Cup Soccer games in 2015, but the times listed on the FIFA website 
were all in times local to the venues. In this exercise you'll help me set the timezones, then 
in the next exercise you'll help me figure out what time I needed to tune in to watch them.

In [2]:
library(lubridate)

# Game2: CAN vs NZL in Edmonton
game2 <- mdy_hm("June 11 2015 19:00")

# Game3: CHN vs NZL in Winnipeg
game3 <- mdy_hm("June 15 2015 18:30")

# Set the timezone to "America/Edmonton"
game2_local <- force_tz(game2, tzone = "America/Edmonton")
game2_local

# Set the timezone to "America/Winnipeg"
game3_local <- force_tz(game3, tzone = "America/Winnipeg")
game3_local

# How long does the team have to rest?
as.period(game2_local %--% game3_local)


Attaching package: 'lubridate'

The following object is masked from 'package:base':

    date



[1] "2015-06-11 19:00:00 MDT"

[1] "2015-06-15 18:30:00 CDT"

[1] "3d 22H 30M 0S"

### Viewing in a timezone
To view a datetime in another timezone use with_tz(). The syntax of with_tz() is the same as force_tz(), 
passing a datetime and set the tzone argument to the desired timezone. Unlike force_tz(), with_tz() isn't 
changing the underlying moment of time, just how it is displayed.
For example, the difference between now() displayed in the "America/Los_Angeles" timezone and 
"Pacific/Auckland" timezone is 0:

now <- now()

with_tz(now, "America/Los_Angeles") - with_tz(now,  "Pacific/Auckland")

Help me figure out when to tune into the games from the previous exercise.

In [3]:
# Most fans will tune in from New Zealand. Use with_tz() to display game2_local in New Zealand time. 
# New Zealand is in the "Pacific/Auckland" timezone.
# What time is game2_local in NZ?
with_tz(game2_local, tzone = "Pacific/Auckland")

# I'll be in Corvallis, Oregon. Use with_tz() to display game2_local my time. 
# Corvallis is in the "America/Los_Angeles" timezone.
# What time is game2_local in Corvallis, Oregon?
with_tz(game2_local, tzone = "America/Los_Angeles")

# Finally, use with_tz() to display game3_local in New Zealand time.
# What time is game3_local in NZ?
with_tz(game3_local, tzone = "Pacific/Auckland")

Time difference of 0 secs

[1] "2015-06-12 13:00:00 NZST"

[1] "2015-06-11 18:00:00 PDT"

[1] "2015-06-16 11:30:00 NZST"

In [5]:
# use dplyr to pipe data 
library(dplyr)
# Use read.csv() to import Auckland daily weather.csv
akl_hourly_raw <- read.csv('Auckland hourly weather.csv')

# Use make_date() and mutate() to combine year, month and mday 
akl_hourly  <- akl_hourly_raw  %>% 
  mutate(date = make_date(year = year, month = month, day = mday))

# Parse datetime_string 
akl_hourly <- akl_hourly  %>% 
  mutate(
    datetime_string = paste(date, time, sep = "T"),
    datetime = ymd_hms(datetime_string)
  )

# Examine datetime and date_utc columns
head(akl_hourly$datetime)
head(akl_hourly$date_utc)
  
# Force datetime to Pacific/Auckland
akl_hourly <- akl_hourly %>%
  mutate(
    datetime = force_tz(datetime, tzone = "Pacific/Auckland"))

# Reexamine datetime
head(akl_hourly$datetime)

# Now tabulate up the difference between the datetime and date_utc columns. It should be zero if our hypothesis was correct.
# Are datetime and date_utc the same moments
# table(akl_hourly$datetime - akl_hourly$date_utc)
# there is a problem with the letters T and Z in date_utc, so table(X - X) expected output is different. 
# Nevertheless, the difference are zero. as we expected  

[1] "2016-01-01 00:00:00 UTC" "2016-01-01 00:30:00 UTC"
[3] "2016-01-01 01:00:00 UTC" "2016-01-01 01:30:00 UTC"
[5] "2016-01-01 02:00:00 UTC" "2016-01-01 02:30:00 UTC"

[1] 2015-12-31T11:00:00Z 2015-12-31T11:30:00Z 2015-12-31T12:00:00Z
[4] 2015-12-31T12:30:00Z 2015-12-31T13:00:00Z 2015-12-31T13:30:00Z
17452 Levels: 2015-12-31T11:00:00Z 2015-12-31T11:30:00Z ... 2016-12-31T10:30:00Z

[1] "2016-01-01 00:00:00 NZDT" "2016-01-01 00:30:00 NZDT"
[3] "2016-01-01 01:00:00 NZDT" "2016-01-01 01:30:00 NZDT"
[5] "2016-01-01 02:00:00 NZDT" "2016-01-01 02:30:00 NZDT"

### Fast parsing with fasttime
The fasttime package provides a single function fastPOSIXct(), designed to read in datetimes 
formatted according to ISO 8601. Because it only reads in one format, and doesn't have to guess a format, 
it is really fast!
You'll see how fast in this exercise by comparing how fast it reads in the dates from the Auckland hourly 
weather data (over 17,000 dates) to lubridates ymd_hms().
To compare run times you'll use the microbenchmark() function from the package of the same name. 
You pass in as many arguments as you want each being an expression to time.

In [13]:
library(microbenchmark)
# install.packages("fasttime")
library(fasttime)

dates = akl_hourly$date_utc

# Examine structure of dates
str(dates)

# Parse dates with fasttime and pipe to str() to verify fastPOSIXct parses them correctly.
fastPOSIXct(dates) %>% str()

# Compare speed of fastPOSIXct() to ymd_hms()
microbenchmark(
  ymd_hms = ymd_hms(dates),
  fasttime = fastPOSIXct(dates),
  times = 20)

# You should see fasttime is about 20 times faster than ymd_hms()

 Factor w/ 17452 levels "2015-12-31T11:00:00Z",..: 1 2 3 4 5 6 7 8 9 10 ...
 POSIXct[1:17454], format: "2015-12-31 12:00:00" "2015-12-31 12:30:00" "2015-12-31 13:00:00" ...


expr,time
ymd_hms,13877501
fasttime,1115202
fasttime,1020300
fasttime,991301
fasttime,965201
fasttime,939702
ymd_hms,10729201
fasttime,966101
ymd_hms,10843502
ymd_hms,11155501


### Outputting pretty dates and times
An easy way to output dates is to use the stamp() function in lubridate. stamp() takes a string 
which should be an example of how the date should be formatted, and returns a function that can be used to format dates
In this exercise you'll practice outputting today() in a nice way.

In [21]:
# Create a stamp based on "Saturday, Jan 1, 2000"
date_stamp <- stamp("Saturday, Jan 1, 2000")

# Print date_stamp
date_stamp

# Call date_stamp on today
date_stamp(today())

# Create and call a stamp based on "12/31/1999"
stamp("12/31/1999")(today())

finished = "I finished 'Dates and Times in R' on Thursday, September 4, 2017!"

# Use string finished for stamp()
stamp(finished)(today())


Multiple formats matched: "%A, %b %d, %Y"(1), "Saturday, Jan %Om, %Y"(1), "Saturday, %Om %d, %Y"(1), "Saturday, %b %d, %Y"(1), "Saturday, Jan %m, %Y"(1), "%A, Jan %Om, %Y"(0), "%A, %Om %d, %Y"(0), "%A, Jan %m, %Y"(0)
Using: "%A, %b %d, %Y"


function (x, locale = "English_United States.1252") 
{
    {
        old_lc_time <- Sys.getlocale("LC_TIME")
        if (old_lc_time != locale) {
            on.exit(Sys.setlocale("LC_TIME", old_lc_time))
            Sys.setlocale("LC_TIME", locale)
        }
    }
    format(x, format = "%A, %b %d, %Y")
}
<environment: 0x0000000005013b00>

[1] "Sunday, Dec 27, 2020"

Multiple formats matched: "%Om/%d/%Y"(1), "%m/%d/%Y"(1)
Using: "%Om/%d/%Y"


[1] "12/27/2020"

Multiple formats matched: "I finished 'Dates and Times in R' on %A, %B %d, %Y!"(1), "I finished 'Dates and Times in R' on Thursday, September %Om, %Y!"(1), "I finished 'Dates and Times in R' on Thursday, %Om %d, %Y!"(1), "I finished 'Dates and Times in R' on Thursday, %B %d, %Y!"(1), "I finished 'Dates and Times in R' on Thursday, September %m, %Y!"(1), "I finished 'Dates and Times in R' on %A, September %Om, %Y!"(0), "I finished 'Dates and Times in R' on %A, %Om %d, %Y!"(0), "I finished 'Dates and Times in R' on %A, September %m, %Y!"(0)
Using: "I finished 'Dates and Times in R' on %A, %B %d, %Y!"


[1] "I finished 'Dates and Times in R' on Sunday, December 27, 2020!"